In [1]:
import pandas as pd
import json
import os
import requests
import gzip
import shutil
import xml.etree.ElementTree as ET
import csv

# Download and extract CCVE DATA

REMEMBER TO DOWNLOAD ANY EXTRA FILES LIKE RECECENT FROM:
https://nvd.nist.gov/vuln/data-feeds

In [4]:
def download_and_extract_cve_data(start_year=2002, end_year=2024, base_url="https://nvd.nist.gov/feeds/json/cve/1.1/"):
    for year in range(start_year, end_year + 1):
        # Construct the download URL for each year
        url = f"{base_url}nvdcve-1.1-{year}.json.gz"
        folder_name = f"../data/CVEs/{str(year)}"
        file_name = f"nvdcve-1.1-{year}.json.gz"
        output_json = f"nvdcve-1.1-{year}.json"
        
        # Create a folder for each year if it doesn't exist
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        
        # Download the file
        print(f"Downloading {file_name}...")
        response = requests.get(url, stream=True)
        
        # Save the .gz file
        gz_path = os.path.join(folder_name, file_name)
        with open(gz_path, 'wb') as gz_file:
            gz_file.write(response.content)
        
        # Unzip the file
        print(f"Extracting {file_name}...")
        json_path = os.path.join(folder_name, output_json)
        with gzip.open(gz_path, 'rb') as f_in:
            with open(json_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        
        print(f"Extracted {output_json} to folder {folder_name}.\n")
    
    print("Download and extraction completed.")

# Example usage:
download_and_extract_cve_data()

Extracting nvdcve-1.1-2002.json.gz...
Extracted nvdcve-1.1-2002.json to folder ../data/CVEs/2002.

Extracting nvdcve-1.1-2003.json.gz...
Extracted nvdcve-1.1-2003.json to folder ../data/CVEs/2003.

Extracting nvdcve-1.1-2004.json.gz...
Extracted nvdcve-1.1-2004.json to folder ../data/CVEs/2004.

Extracting nvdcve-1.1-2005.json.gz...
Extracted nvdcve-1.1-2005.json to folder ../data/CVEs/2005.

Extracting nvdcve-1.1-2006.json.gz...
Extracted nvdcve-1.1-2006.json to folder ../data/CVEs/2006.

Extracting nvdcve-1.1-2007.json.gz...
Extracted nvdcve-1.1-2007.json to folder ../data/CVEs/2007.

Extracting nvdcve-1.1-2008.json.gz...
Extracted nvdcve-1.1-2008.json to folder ../data/CVEs/2008.

Extracting nvdcve-1.1-2009.json.gz...
Extracted nvdcve-1.1-2009.json to folder ../data/CVEs/2009.

Extracting nvdcve-1.1-2010.json.gz...
Extracted nvdcve-1.1-2010.json to folder ../data/CVEs/2010.

Extracting nvdcve-1.1-2011.json.gz...
Extracted nvdcve-1.1-2011.json to folder ../data/CVEs/2011.

Extracting

# Convert CPE data from XML to CSV

In [8]:

def process_cpe_data(xml_file, output_csv):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Extract the timestamp from the "generator" section
    generator = root.find(".//{http://cpe.mitre.org/dictionary/2.0}generator")
    release_date = generator.find("{http://cpe.mitre.org/dictionary/2.0}timestamp").text if generator is not None else "N/A"
    
    # Open the CSV file for writing
    with open(output_csv, mode='w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['cpe_name', 'title', 'notes', 'references', 'deprecated', 'deprecation_date', 'release_date']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write the header row
        writer.writeheader()

        # Iterate over each "cpe-item" element in the XML
        for cpe_item in root.findall(".//{http://cpe.mitre.org/dictionary/2.0}cpe-item"):
            # Extract the "name" attribute
            cpe_name = cpe_item.attrib.get('name', 'N/A')

            # Extract "title" elements (there can be multiple titles in different languages)
            titles = cpe_item.findall(".//{http://cpe.mitre.org/dictionary/2.0}title")
            title_texts = [title.text for title in titles if title.text]  # Collect all titles

            # Extract "notes" elements
            notes = cpe_item.findall(".//{http://cpe.mitre.org/dictionary/2.0}notes")
            notes_texts = [note.text for note in notes if note.text]  # Collect all notes

            # Extract "references" elements
            references = cpe_item.findall(".//{http://cpe.mitre.org/dictionary/2.0}reference")
            reference_texts = [ref.attrib.get('href', 'N/A') for ref in references]  # Collect reference hrefs

            # Extract "deprecated" attribute and "deprecation_date" if present
            deprecated = cpe_item.attrib.get('deprecated', 'false')
            deprecation_date = cpe_item.attrib.get('deprecation_date', 'N/A')

            # Write the data to CSV
            writer.writerow({
                'cpe_name': cpe_name,
                'title': ', '.join(title_texts),
                'notes': ', '.join(notes_texts),
                'references': ', '.join(reference_texts),
                'deprecated': deprecated,
                'deprecation_date': deprecation_date,
                'release_date': release_date
            })
    
    print(f"Data successfully written to {output_csv}")

# Example usage:
process_cpe_data('../data/cpe_data_xml/official-cpe-dictionary_v2.3.xml', '../data/CPEs/cpe_data.csv')


Data successfully written to ../data/CPEs/cpe_data.csv


# Process the json CVE Data

In [9]:
import os
import json
import csv

def process_cve_data_per_json(json_root_dir, output_dir):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Walk through the directory structure
    for root, dirs, files in os.walk(json_root_dir):
        for file in files:
            if file.endswith(".json"):
                json_file_path = os.path.join(root, file)
                print(f"Processing {json_file_path}...")
                
                # Create a CSV filename based on the JSON file name
                csv_filename = f"{os.path.splitext(file)[0]}.csv"
                csv_file_path = os.path.join(output_dir, csv_filename)
                
                # Open the CSV file for writing
                with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
                    fieldnames = ['CVE_ID', 'Timestamp', 'CPE_URI', 'Version_Start', 'Version_End', 'CWE', 'Impact_BaseScore', 'Impact_Severity', 'Published_Date', 'Last_Modified_Date']
                    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                    
                    # Write the header row
                    writer.writeheader()

                    # Process the JSON file
                    with open(json_file_path, 'r') as f:
                        cve_data = json.load(f)

                        # Extract CVE data from each "CVE_Items" entry
                        for item in cve_data.get('CVE_Items', []):
                            cve_id = item['cve']['CVE_data_meta']['ID']
                            timestamp = cve_data['CVE_data_timestamp']

                            # Extract CWE(s) from the problemtype section
                            cwes = []
                            for problemtype_data in item['cve']['problemtype']['problemtype_data']:
                                for desc in problemtype_data.get('description', []):
                                    cwes.append(desc.get('value', 'N/A'))
                            cwe_list = ', '.join(cwes) if cwes else 'None'

                            # Extract matched CPEs from the configurations section
                            for node in item.get('configurations', {}).get('nodes', []):
                                for cpe_match in node.get('cpe_match', []):
                                    cpe_uri = cpe_match.get('cpe23Uri', 'N/A')
                                    version_start = cpe_match.get('versionStartIncluding', 'N/A')
                                    version_end = cpe_match.get('versionEndIncluding', 'N/A')
                                    
                                    # Extract impact data (base score and severity)
                                    base_score = 'N/A'
                                    severity = 'N/A'
                                    impact_v3 = item.get('impact', {}).get('baseMetricV3', {})
                                    if impact_v3:
                                        base_score = impact_v3.get('cvssV3', {}).get('baseScore', 'N/A')
                                        severity = impact_v3.get('cvssV3', {}).get('baseSeverity', 'N/A')
                                    
                                    # Extract publishedDate and lastModifiedDate
                                    published_date = item.get('publishedDate', 'N/A')
                                    last_modified_date = item.get('lastModifiedDate', 'N/A')

                                    # Write each CPE entry as a separate row
                                    writer.writerow({
                                        'CVE_ID': cve_id,
                                        'Timestamp': timestamp,
                                        'CPE_URI': cpe_uri,
                                        'Version_Start': version_start,
                                        'Version_End': version_end,
                                        'CWE': cwe_list,
                                        'Impact_BaseScore': base_score,
                                        'Impact_Severity': severity,
                                        'Published_Date': published_date,
                                        'Last_Modified_Date': last_modified_date
                                    })

                print(f"Data for {json_file_path} written to {csv_file_path}")



In [10]:
def preview_csv(file_path):
    # Read the CSV file in chunks to handle large files
    chunk_size = 1000  # Adjust based on memory availability
    df_chunk = pd.read_csv(file_path, chunksize=chunk_size)
    
    # Load only the first chunk to inspect the data
    df = next(df_chunk)
    
    # Display the column names and first three rows
    print("Columns:", df.columns)
    print("\nFirst 3 rows:")
    display(df.head(3))


In [11]:
import json

def preview_cwe_json(file_path):
    # Open the JSON file and load the entire data
    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
            
            # Check if the data is a list of CWEs
            if isinstance(data, list) and len(data) > 0:
                # Preview the first three entries
                print("Previewing the first 3 CWE entries:")
                for entry in data[:3]:
                    print(f"\nCWE-ID: {entry.get('CWE-ID', 'N/A')}")
                    print(f"Name: {entry.get('Name', 'N/A')}")
                    print(f"Description: {entry.get('Description', 'N/A')}")
                    print(f"Common Consequences:")
                    for consequence in entry.get('Common_Consequences', []):
                        print(f"  Scope: {consequence.get('Scope', 'N/A')}")
                        print(f"  Impacts: {', '.join(consequence.get('Impacts', []))}")
                    print(f"Related Weaknesses:")
                    for weakness in entry.get('Related_Weaknesses', []):
                        print(f"  Nature: {weakness.get('Nature', 'N/A')}")
                        print(f"  CWE_ID: {weakness.get('CWE_ID', 'N/A')}")
            else:
                print("The JSON data is not a list or is empty.")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")


In [13]:
cpe_data = '../data/CPEs/cpe_data.csv'
cve_data = '../data/CVEs/cveData.json'
cwe_data = '../data/CWEs/cweData.json'

In [12]:
# process_cve_data('../data/CVEs/', '../data/CVEs/cve_data_output.csv')
process_cve_data_per_json('../data/CVEs/', '../data/CVEs/CSVs')

Processing ../data/CVEs/2009/nvdcve-1.1-2009.json...
Data for ../data/CVEs/2009/nvdcve-1.1-2009.json written to ../data/CVEs/CSVs/nvdcve-1.1-2009.csv
Processing ../data/CVEs/modified/nvdcve-1.1-modified.json...
Data for ../data/CVEs/modified/nvdcve-1.1-modified.json written to ../data/CVEs/CSVs/nvdcve-1.1-modified.csv
Processing ../data/CVEs/2022/nvdcve-1.1-2022.json...
Data for ../data/CVEs/2022/nvdcve-1.1-2022.json written to ../data/CVEs/CSVs/nvdcve-1.1-2022.csv
Processing ../data/CVEs/2010/nvdcve-1.1-2010.json...
Data for ../data/CVEs/2010/nvdcve-1.1-2010.json written to ../data/CVEs/CSVs/nvdcve-1.1-2010.csv
Processing ../data/CVEs/2020/nvdcve-1.1-2020.json...
Data for ../data/CVEs/2020/nvdcve-1.1-2020.json written to ../data/CVEs/CSVs/nvdcve-1.1-2020.csv
Processing ../data/CVEs/2011/nvdcve-1.1-2011.json...
Data for ../data/CVEs/2011/nvdcve-1.1-2011.json written to ../data/CVEs/CSVs/nvdcve-1.1-2011.csv
Processing ../data/CVEs/2008/nvdcve-1.1-2008.json...
Data for ../data/CVEs/2008/

In [14]:

df = pd.read_csv(cpe_data)
df.head(3)


,cpe_name,title,notes,references,deprecated,deprecation_date,release_date
0,cpe:/a:%240.99_kindle_books_project:%240.99_ki...,$0.99 Kindle Books project $0.99 Kindle Books ...,NaN,https://play.google.com/store/apps/details?id=...,False,NaN,2024-12-07T04:51:54.238Z
1,cpe:/a:%40nubosoftware%2fnode-static_project:%...,@nubosoftware/node-static Project @nubosoftwar...,NaN,https://www.npmjs.com/package/@nubosoftware/no...,False,NaN,2024-12-07T04:51:54.238Z
2,cpe:/a:%40thi.ng%2fegf_project:%40thi.ng%2fegf...,@thi.ng/egf Project @thi.ng/egf for Node.js,NaN,https://github.com/thi-ng/umbrella/security/ad...,False,NaN,2024-12-07T04:51:54.238Z


In [15]:
len(df)

1339423

In [13]:
preview_cwe_json(cve_data)

The JSON data is not a list or is empty.


# CREATE sqlite DATABASE 

In [1]:
import os
import json
import sqlite3

# Step 1: Define the database schema
db_path = "cve_database.db"  # SQLite database file
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create the cpe table
cursor.execute("""
CREATE TABLE IF NOT EXISTS cpe (
    cpe_id TEXT PRIMARY KEY
)
""")

# Create the cve table
cursor.execute("""
CREATE TABLE IF NOT EXISTS cve (
    cpe_id TEXT,
    cve_id TEXT,
    cve_data TEXT,
    PRIMARY KEY (cpe_id, cve_id),
    FOREIGN KEY (cpe_id) REFERENCES cpe (cpe_id)
)
""")
conn.commit()

# Step 2: Process JSON files
def process_json(file_path):
    """Extract CPE and CVE information from a JSON file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    cpe_cve_map = {}
    for item in data.get("CVE_Items", []):
        cve_id = item.get("cve", {}).get("CVE_data_meta", {}).get("ID", "Unknown")
        
        # Collect the full CVE object as a JSON string
        cve_full_data = json.dumps(item)  # Store the entire CVE object

        # Extract CPEs
        configurations = item.get("configurations", {}).get("nodes", [])
        for node in configurations:
            for cpe in node.get("cpe_match", []):
                cpe_uri = cpe.get("cpe23Uri")
                if cpe_uri:
                    if cpe_uri not in cpe_cve_map:
                        cpe_cve_map[cpe_uri] = []
                    # Add only unique CVE IDs for each CPE
                    if cve_id not in [entry["cve_id"] for entry in cpe_cve_map[cpe_uri]]:
                        cpe_cve_map[cpe_uri].append({
                            "cve_id": cve_id,
                            "cve_data": cve_full_data
                        })
    
    return cpe_cve_map

def save_to_database(cpe_cve_map):
    """Save extracted CPE-CVE mappings to the SQLite database."""
    for cpe, cves in cpe_cve_map.items():
        # Insert the CPE into the cpe table
        cursor.execute("""
        INSERT OR IGNORE INTO cpe (cpe_id)
        VALUES (?)
        """, (cpe,))
        
        # Insert the CVEs into the cve table
        for cve_entry in cves:
            cursor.execute("""
            INSERT OR IGNORE INTO cve (cpe_id, cve_id, cve_data)
            VALUES (?, ?, ?)
            """, (cpe, cve_entry["cve_id"], cve_entry["cve_data"]))
    conn.commit()

# Step 3: Iterate through all JSON files
base_dir = "../data/CVEs"  # Directory containing CVE JSON files
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)
            print(f"Processing file: {file_path}")
            cpe_cve_map = process_json(file_path)
            save_to_database(cpe_cve_map)

# Step 4: Close the database connection
conn.close()
print("Database created and populated successfully!")





Processing file: ../data/CVEs/nvdcve-1.1-modified.json
Processing file: ../data/CVEs/nvdcve-1.1-recent.json
Processing file: ../data/CVEs/2009/nvdcve-1.1-2009.json
Processing file: ../data/CVEs/2022/nvdcve-1.1-2022.json
Processing file: ../data/CVEs/2010/nvdcve-1.1-2010.json
Processing file: ../data/CVEs/2020/nvdcve-1.1-2020.json
Processing file: ../data/CVEs/2011/nvdcve-1.1-2011.json
Processing file: ../data/CVEs/2008/nvdcve-1.1-2008.json
Processing file: ../data/CVEs/2005/nvdcve-1.1-2005.json
Processing file: ../data/CVEs/2002/nvdcve-1.1-2002.json
Processing file: ../data/CVEs/2013/nvdcve-1.1-2013.json
Processing file: ../data/CVEs/2015/nvdcve-1.1-2015.json
Processing file: ../data/CVEs/2003/nvdcve-1.1-2003.json
Processing file: ../data/CVEs/2024/nvdcve-1.1-2024.json
Processing file: ../data/CVEs/2014/nvdcve-1.1-2014.json
Processing file: ../data/CVEs/2006/nvdcve-1.1-2006.json
Processing file: ../data/CVEs/2007/nvdcve-1.1-2007.json
Processing file: ../data/CVEs/2021/nvdcve-1.1-2021.j

# UPDATE DATABASE WITH IN BETWEEN CVES

In [1]:
import os
import json
import sqlite3
from packaging.version import Version, InvalidVersion

# Connect to the database
db_path = "cve_database.db"
conn = sqlite3.connect(db_path, timeout=30)  # Set a longer timeout for locking
conn.execute("PRAGMA journal_mode=WAL;")
cursor = conn.cursor()

def parse_version(version_str):
    """Parse a version string into a comparable format. Return None if invalid."""
    try:
        return Version(version_str)
    except InvalidVersion:
        return None

def is_version_in_range(version, start_including=None, end_excluding=None):
    """Check if a version is within a given range."""
    version_obj = parse_version(version)
    if not version_obj:
        return False  # If version cannot be parsed, return False

    if start_including:
        start_version = parse_version(start_including)
        if start_version and version_obj < start_version:
            return False
    if end_excluding:
        end_version = parse_version(end_excluding)
        if end_version and version_obj >= end_version:
            return False
    return True


def process_json_for_version_ranges(file_path):
    """Identify CVEs with version ranges and update the database."""
    # Open a new database connection for this function
    local_conn = sqlite3.connect(db_path)
    local_cursor = local_conn.cursor()

    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for item in data.get("CVE_Items", []):
        cve_id = item.get("cve", {}).get("CVE_data_meta", {}).get("ID", "Unknown")
        full_cve_data = json.dumps(item)  # Store the full CVE object

        configurations = item.get("configurations", {}).get("nodes", [])
        for node in configurations:
            for cpe in node.get("cpe_match", []):
                cpe_uri = cpe.get("cpe23Uri")
                version_start = cpe.get("versionStartIncluding")
                version_end = cpe.get("versionEndExcluding")

                if cpe_uri and (version_start or version_end):
                    # Extract the base CPE name (everything before version info)
                    base_cpe = ":".join(cpe_uri.split(":")[:5])

                    # Query the database for all matching CPEs
                    local_cursor.execute("SELECT cpe_id FROM cpe WHERE cpe_id LIKE ?", (f"{base_cpe}%",))
                    matching_cpes = local_cursor.fetchall()

                    for matching_cpe_row in matching_cpes:
                        matching_cpe = matching_cpe_row[0]
                        # Extract the version from the matching CPE
                        cpe_parts = matching_cpe.split(":")
                        if len(cpe_parts) > 5:  # Ensure version exists
                            version = cpe_parts[5]
                            if is_version_in_range(version, version_start, version_end):
                                # Add the CVE to the database for this CPE
                                local_cursor.execute("""
                                INSERT OR IGNORE INTO cve (cpe_id, cve_id, cve_data)
                                VALUES (?, ?, ?)
                                """, (matching_cpe, cve_id, full_cve_data))
    local_conn.commit()
    local_conn.close()


# Directory containing JSON files
base_dir = "../data/CVEs" 

# Process all JSON files in the directory
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)
            print(f"Processing file for version ranges: {file_path}")
            process_json_for_version_ranges(file_path)

# Close the database connection
conn.close()
print("Database updated successfully for version ranges.")


Processing file for version ranges: ../data/CVEs/nvdcve-1.1-modified.json
Processing file for version ranges: ../data/CVEs/nvdcve-1.1-recent.json
Processing file for version ranges: ../data/CVEs/2009/nvdcve-1.1-2009.json
Processing file for version ranges: ../data/CVEs/2022/nvdcve-1.1-2022.json
Processing file for version ranges: ../data/CVEs/2010/nvdcve-1.1-2010.json
Processing file for version ranges: ../data/CVEs/2020/nvdcve-1.1-2020.json
Processing file for version ranges: ../data/CVEs/2011/nvdcve-1.1-2011.json
Processing file for version ranges: ../data/CVEs/2008/nvdcve-1.1-2008.json
Processing file for version ranges: ../data/CVEs/2005/nvdcve-1.1-2005.json
Processing file for version ranges: ../data/CVEs/2002/nvdcve-1.1-2002.json
Processing file for version ranges: ../data/CVEs/2013/nvdcve-1.1-2013.json
Processing file for version ranges: ../data/CVEs/2015/nvdcve-1.1-2015.json
Processing file for version ranges: ../data/CVEs/2003/nvdcve-1.1-2003.json
Processing file for version r

TEST

In [21]:
import sqlite3
import json

# Connect to the database
conn = sqlite3.connect("cve_database.db")
cursor = conn.cursor()

# Query for a specific CPE
example_cpe = "cpe:2.3:a:busybox:busybox:*:*:*:*:*:*:*:*"
cursor.execute("SELECT cve_id, cve_data FROM cve WHERE cpe_id = ?", (example_cpe,))
results = cursor.fetchall()

if results:
    print(f"CVEs for {example_cpe}:")
    for cve_id, cve_data in results:
        print(f"\nCVE ID: {cve_id}")
        print(f"CVE Data: {json.dumps(json.loads(cve_data), indent=4)}")  # Pretty print
else:
    print(f"No CVEs found for {example_cpe}.")

conn.close()



CVEs for cpe:2.3:a:busybox:busybox:*:*:*:*:*:*:*:*:

CVE ID: CVE-2011-2716
CVE Data: {
    "cve": {
        "data_type": "CVE",
        "data_format": "MITRE",
        "data_version": "4.0",
        "CVE_data_meta": {
            "ID": "CVE-2011-2716",
            "ASSIGNER": "secalert@redhat.com"
        },
        "problemtype": {
            "problemtype_data": [
                {
                    "description": [
                        {
                            "lang": "en",
                            "value": "CWE-20"
                        }
                    ]
                }
            ]
        },
        "references": {
            "reference_data": [
                {
                    "url": "http://secunia.com/advisories/45363",
                    "name": "45363",
                    "refsource": "SECUNIA",
                    "tags": [
                        "Vendor Advisory"
                    ]
                },
                {
                   

In [5]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("cve_database.db")
cursor = conn.cursor()

# Query for a specific CPE
example_cpe = "cpe:2.3:a:busybox:busybox:1.36.1:*:*:*:*:*:*:*"
cursor.execute("SELECT cve_id FROM cve WHERE cpe_id = ?", (example_cpe,))
results = cursor.fetchall()

if results:
    print(f"CVEs for {example_cpe}:")
    count = 0
    for cve_id, in results:  # Unpacking the single-column result
        count +=1
        print(f"- {cve_id}")
    print(f"no of cves: {count}")
else:
    print(f"No CVEs found for {example_cpe}.")

conn.close()


CVEs for cpe:2.3:a:busybox:busybox:1.36.1:*:*:*:*:*:*:*:
- CVE-2021-28831
- CVE-2021-42374
- CVE-2021-42378
- CVE-2021-42379
- CVE-2021-42380
- CVE-2021-42381
- CVE-2021-42382
- CVE-2021-42384
- CVE-2021-42385
- CVE-2021-42386
- CVE-2023-42363
- CVE-2023-42364
- CVE-2023-42365
- CVE-2023-42366
no of cves: 14
